In [6]:
import pandas as pd 

In [7]:
df = pd.read_excel('tables/violation_rates.xlsx')

In [8]:
df_sum =df.describe()

In [9]:
df_sum.to_excel('tables/summary_statistics.xlsx')

In [10]:
print(df_sum.to_latex(float_format="%.4f"))

\begin{tabular}{lrrr}
\toprule
{} &   RoCoF &     AFV &      AV \\
\midrule
count & 35.0000 & 35.0000 & 35.0000 \\
mean  &  0.1462 &  0.2558 &  0.2947 \\
std   &  0.0066 &  0.0036 &  0.0047 \\
min   &  0.1242 &  0.2470 &  0.2781 \\
25\%   &  0.1431 &  0.2542 &  0.2939 \\
50\%   &  0.1486 &  0.2550 &  0.2951 \\
75\%   &  0.1497 &  0.2572 &  0.2958 \\
max   &  0.1525 &  0.2651 &  0.3034 \\
\bottomrule
\end{tabular}



In [1]:
from powerNetworkSolver import PowerNetworkSolver
from powerNetwork import networkTransform, getGenerators, getBuses, getLines,getNumBuses, getNumLines
from powerNetwork import getUndGraph, kron_reduction, edge_removing
from disturbancesGnr import normaldisturbances
from violationChecking import globalcheck
import seaborn as sns; sns.set_theme()
#importing the function for multi-edge removal
from powerNetwork import multi_edge_removing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
from numpy.linalg import inv 
import networkx as nx
from numpy.random import multivariate_normal
from disturbancesGnr import normaldisturbances
from scipy.integrate import odeint
from violationChecking import violationcheck,globalcheck,globalchecksubset
import time 
import math 
from functools import reduce
from copy import deepcopy


In [2]:
from greedy_algorithm import greedy_algorithm

In [3]:
from pypower.api import case39

In [4]:
network39, subnetwork39 = networkTransform(case39())
df_lines39 = getLines(subnetwork39)
df_buses39 = getBuses(subnetwork39)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [5]:
n39, ngnr39, unG39 = getUndGraph(df_buses39,df_lines39, network39)
A39, redL39, redA39  = kron_reduction(n39, ngnr39, unG39)
node_list =  np.arange(ngnr39)+1

#the previous adjacent matrix 
# redA39 = np.array([[0,2.484, 2.858, 2.372, 1.031, 0.222,2.13,13.057,3.676,7.834],
#               [2.484,0,10.494,1.781,0.774,0.166,1.599,1.473,0.903,5.885],
#               [2.858,10.494,0,2.522,1.096,0.236,2.264,1.729,1.144,4.772],
#               [2.372,1.781,2.522,0,17.136,0.535,5.14,1.699,1.755,1.367],
#               [1.031,0.774, 1.096,17.136,0 ,0.232, 2.235, 0.739, 0.763, 0.594],
#               [0.222,0.166,0.236,0.535,0.232,0,1.574, 0.159, 0.164, 0.128],
#               [2.13,1.599,2.264, 5.14, 2.235, 1.574, 0, 1.526, 1.576, 1.227],
#               [13.057,1.473,1.729, 1.699,0.739, 0.159, 1.526, 0, 4.283, 4.159],
#               [3.676, 0.903, 1.144, 1.755, 0.763, 0.164, 1.576, 4.283, 0, 1.37],
#               [7.834, 5.885, 4.772, 1.367, 0.594, 0.128, 1.227, 4.159, 1.37 ,0]])

# redA39 = np.ones((ngnr39,ngnr39))
# np.fill_diagonal(redA39, 0)
 

In [6]:
K = 1
M = np.array([0.2228, 0.1607, 0.1899, 0.1517, 0.1379, 0.1846, 0.1401, 0.1289, 0.183, 2.6526])
D = np.array([0.0332,0.076,0.0862,0.0838,0.0674,0.0862,0.0743,0.0716,0.1101,0.1333])
# Ome = np.zeros(n)
pi = math.pi
t =  2
nn = 100
dt = np.linspace(0, t, nn+1)

sigma = 0.01
theta0 = np.zeros(ngnr39)
omega0 = np.zeros(ngnr39)
OMEGA = np.zeros(ngnr39)
model39 = PowerNetworkSolver(theta0,omega0,redA39, ngnr39, D,M,K,OMEGA)

In [7]:
from numpy.random import seed

In [8]:
#generate numerical tracks 
seed(100)
disturbances = normaldisturbances(ngnr39,1,sigma)
sol0 = np.pad(disturbances[0], (ngnr39,0), 'constant', constant_values=(0,0))
single_sol = model39.solkuramoto(sol0,dt)
sol_domega = model39.getDotOmega(single_sol[:,:ngnr39],single_sol[:,ngnr39:],nn)

In [11]:
# simulation to calculate failure rates 
check_times = 100
KK =100#repetition times  
thres = np.array([0.2,2])#thres1 is for omega, thres2 is for omega_dot
# rates39 = model39.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [12]:
un_graph = unG39
int_theta = theta0
int_omega = omega0
max_itr = 5
type_rate = 1

In [13]:
new_graagreedy_algorithm(un_graph,int_theta,int_omega,D,M,K,OMEGA,KK,check_times,sigma,thres,t,nn，normaldisturbances,max_itr,type_rate)

NameError: name 'np' is not defined